In [1]:
import numpy as np
from numpy import linalg as LA
from numpy.linalg import inv
from numpy.linalg import det

In [2]:
def norm(x):
  return LA.norm(x,2)

def is_pos_def(x):
    return np.all(LA.eigvals(x) > 0)
  
def f1(x):
  return -13 + x[0] + ((5 - x[1]) * x[1] - 2) * x[1]

def f2(x):
  return -29 + x[0] + ((x[1] + 1) * x[1] - 14) * x[1]

def f(x):
  return f1(x)*f1(x) + f2(x)*f2(x)

def gradf1(x):
  g1f1 = 1
  g2f1 = -3*pow(x[1],2) + 10*x[1] - 2
  return np.array([g1f1, g2f1], dtype=np.float64)

def gradf2(x):
  g1f2 = 1
  g2f2 = 3*pow(x[1],2) + 2*x[1] - 14
  return np.array([g1f2, g2f2], dtype=np.float64)

def gradf(x):
  return 2*f1(x)*gradf1(x) + 2*f2(x)*gradf2(x)

def h(x):
  h1 = 4
  h2 = 24*x[1] - 32
  h3 = h2
  h4 = 60*pow(x[1],4) - 160*pow(x[1],3) + 24*pow(x[1],2) - 480*x[1] + 24*x[0] + 24
  return np.array([[h1, h2], [h3,h4]], dtype=np.float64)

def J(x):
  return np.array([[gradf1(x)[0], gradf1(x)[1]], [gradf2(x)[0], gradf2(x)[1]]], dtype=np.float64)

def F(x):
  return np.array([f1(x), f2(x)], dtype=np.float64)

In [3]:
def backtracking(x,alpha,beta,s,epsilon):
  iter = 0
  while(norm(gradf(x)) > epsilon):
    iter = iter + 1
    t = s
    v1 = f(x) - f(x - t*gradf(x))
    v2 = alpha * t * norm(gradf(x))**2
    while(v1 < v2):
      t = beta * t
      v1 = f(x) - f(x - t*gradf(x))
      v2 = alpha * t * norm(gradf(x))**2
    x = x - t * gradf(x)
  print("The method converged")
  return iter, x, f(x)

In [4]:
def newton_hybrid(x,alpha,beta,s,epsilon):
  iter = 0
  d = -gradf(x)

  if is_pos_def(h(x)):
    d = LA.solve(h(x), -gradf(x))
  
  while(norm(gradf(x)) > epsilon):
    d = -gradf(x)

    if is_pos_def(h(x)):
      d = LA.solve(h(x), -gradf(x))

    iter = iter + 1
    t = s
    v1 = f(x) - f(x + t*d)
    v2 = -alpha * t * (gradf(x).T @ d)
    while(v1 < v2):
      t = beta * t
      v1 = f(x) - f(x + t*d)
      v2 = -alpha * t * (gradf(x).T @ d)
    x = x + t * d
  print("The method converged")
  return iter, x, f(x)

In [5]:
def damped_gauss_newton(x,alpha,beta,s,epsilon):
  iter = 0
  
  while(norm(gradf(x)) > epsilon):

    if(LA.det(J(x).T @ J(x)) == 0):
      print("J(x_k)^T * J(x_k) is singular")
      print("The method did not converge")
      print("The current values of number of iterations, x and f(x) are:")
      return iter, x, f(x)
      
    temp = inv(J(x).T @ J(x))
    d = (-temp @ J(x).T) @ F(x)
    iter = iter + 1
    t = s
    v1 = f(x) - f(x + t*d)
    v2 = -alpha * t * (gradf(x).T @ d)

    while(v1 < v2):
      t = beta * t
      v1 = f(x) - f(x + t*d)
      v2 = -alpha * t * (gradf(x).T @ d)
    
    x = x + t * d
  print("The method converged")
  return iter, x, f(x)

In [6]:
epsilon = 1e-5
s = 1
alpha = 0.5
beta = 0.5
x1 = np.array([-50,7], dtype=np.float64)
x2 = np.array([20,7], dtype=np.float64)
x3 = np.array([20,-18], dtype=np.float64)
x4 = np.array([5,-10], dtype=np.float64)

### For the gradient method with backtracking

In [7]:
print("For x0 =", x1)
iterb1, xb1, fb1 = backtracking(x1,alpha,beta,s,epsilon)
print("Number of iterations:", iterb1,", x:", xb1, ", f(x):",fb1,"\n")

print("For x0 =", x2)
iterb2, xb2, fb2 = backtracking(x2,alpha,beta,s,epsilon)
print("Number of iterations:", iterb2,", x:", xb2, ", f(x):",fb2,"\n")

print("For x0 =", x3)
iterb3, xb3, fb3 = backtracking(x3,alpha,beta,s,epsilon)
print("Number of iterations:", iterb3,", x:", xb3, ", f(x):",fb3,"\n")

print("For x0 =", x4)
iterb4, xb4, fb4 = backtracking(x4,alpha,beta,s,epsilon)
print("Number of iterations:", iterb4,", x:", xb4, ", f(x):",fb4,"\n")

For x0 = [-50.   7.]
The method converged
Number of iterations: 2252 , x: [4.99999659 4.00000006] , f(x): 1.6909807165885658e-11 

For x0 = [20.  7.]
The method converged
Number of iterations: 2447 , x: [5.00000334 3.99999994] , f(x): 1.6225287498306366e-11 

For x0 = [ 20. -18.]
The method converged
Number of iterations: 2472 , x: [11.41279066 -0.89680456] , f(x): 48.98425367929612 

For x0 = [  5. -10.]
The method converged
Number of iterations: 2123 , x: [5.00000332 3.99999994] , f(x): 1.5976729507672273e-11 



### For the hybrid Newton’s method

In [8]:
print("For x0 =", x1)
iterb1, xb1, fb1 = newton_hybrid(x1,alpha,beta,s,epsilon)
print("Number of iterations:", iterb1,", x:", xb1, ", f(x):",fb1,"\n")

print("For x0 =", x2)
iterb2, xb2, fb2 = newton_hybrid(x2,alpha,beta,s,epsilon)
print("Number of iterations:", iterb2,", x:", xb2, ", f(x):",fb2,"\n")

print("For x0 =", x3)
iterb3, xb3, fb3 = newton_hybrid(x3,alpha,beta,s,epsilon)
print("Number of iterations:", iterb3,", x:", xb3, ", f(x):",fb3,"\n")

print("For x0 =", x4)
iterb4, xb4, fb4 = newton_hybrid(x4,alpha,beta,s,epsilon)
print("Number of iterations:", iterb4,", x:", xb4, ", f(x):",fb4,"\n")

For x0 = [-50.   7.]
The method converged
Number of iterations: 8 , x: [5. 4.] , f(x): 3.2153437889479672e-21 

For x0 = [20.  7.]
The method converged
Number of iterations: 8 , x: [5. 4.] , f(x): 7.573184445518013e-21 

For x0 = [ 20. -18.]
The method converged
Number of iterations: 16 , x: [11.41277899 -0.89680525] , f(x): 48.98425367924001 

For x0 = [  5. -10.]
The method converged
Number of iterations: 13 , x: [11.41277899 -0.89680525] , f(x): 48.98425367924001 



### For the damped Gauss-Newton’s method with a backtracking line search strategy

In [9]:
print("For x0 =", x1)
iterb1, xb1, fb1 = damped_gauss_newton(x1,alpha,beta,s,epsilon)
print("Number of iterations:", iterb1,", x:", xb1, ", f(x):",fb1,"\n")

print("For x0 =", x2)
iterb2, xb2, fb2 = damped_gauss_newton(x2,alpha,beta,s,epsilon)
print("Number of iterations:", iterb2,", x:", xb2, ", f(x):",fb2,"\n")

print("For x0 =", x3)
iterb3, xb3, fb3 = damped_gauss_newton(x3,alpha,beta,s,epsilon)
print("Number of iterations:", iterb3,", x:", xb3, ", f(x):",fb3,"\n")

print("For x0 =", x4)
iterb4, xb4, fb4 = damped_gauss_newton(x4,alpha,beta,s,epsilon)
print("Number of iterations:", iterb4,", x:", xb4, ", f(x):",fb4,"\n")

For x0 = [-50.   7.]
The method converged
Number of iterations: 30 , x: [5. 4.] , f(x): 0.0 

For x0 = [20.  7.]
The method converged
Number of iterations: 29 , x: [5. 4.] , f(x): 1.1233379290347208e-27 

For x0 = [ 20. -18.]
J(x_k)^T * J(x_k) is singular
The method did not converge
The current values of number of iterations, x and f(x) are:
Number of iterations: 28 , x: [12.8788636  -0.89680526] , f(x): 53.28306277109294 

For x0 = [  5. -10.]
J(x_k)^T * J(x_k) is singular
The method did not converge
The current values of number of iterations, x and f(x) are:
Number of iterations: 26 , x: [13.00946764 -0.89680526] , f(x): 54.08308373751153 

